# beomi/KcELECTRA-base

In [1]:
# %run KcELECTRA-base.version.py     # 필요한 라이브러리 설치
# !pip freeze                          # 설치된 라이브러리 확인
# Learning Device list Check 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2034573008267837899
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 82448744448
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8707885697732648248
physical_device_desc: "device: 0, name: NVIDIA H100 80GB HBM3, pci bus id: 0000:00:05.0, compute capability: 9.0"
xla_global_id: 416903419
]


In [2]:
import re
import torch
import pandas as pd

# GPU 설정
# 없을 시 CPU
# CPU로 뜨지만, GPU 잘만 돌아감
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device:", device)

device: cuda:0


In [3]:
# 엑셀 파일에서 데이터프레임 읽기
df = pd.read_excel(r"C:\Users\GJAISCHOOL\Desktop\X_filter\Algorithm\dataset\kc_Sentence_data.xlsx")

print(df.shape)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\GJAISCHOOL\\Desktop\\X_filter\\Algorithm\\dataset\\sample_data(500).xlsx'

In [4]:
null_idx = df[df.label.isnull()].index                             # 해당 index에 null 값 확인
df.loc[null_idx, "Sentence"]                                       # null 값이 존재한 인덱스의 content 값 불러오기

# lable은 content의 가장 끝 문장열로 설정
df.loc[null_idx, "label"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-1])

# content는 "\t" 앞부분까지의 문자열로 설정
df.loc[null_idx, "Sentence"] = df.loc[null_idx, "Sentence"].apply(lambda x: x[-2])

In [5]:
train_data = df.sample(frac=0.8, random_state=42)                 # train(80%), test(20%) 셋 구분 
test_data = df.drop(train_data.index)                             # 랜덤으로 샘플링(랜덤으로 숫자 배치)

# 데이터셋 갯수 확인
print('중복 제거 전 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 전 테스트 데이터셋 : {}'.format(len(test_data)))

# 중복 데이터 제거
train_data.drop_duplicates(subset=["Sentence"], inplace=True)
test_data.drop_duplicates(subset=["Sentence"], inplace=True)

# 데이터셋 갯수 확인
print('중복 제거 후 학습 데이터셋 : {}'.format(len(train_data)))
print('중복 제거 후 테스트 데이터셋 : {}'.format(len(test_data)))

중복 제거 전 학습 데이터셋 : 407
중복 제거 전 테스트 데이터셋 : 102
중복 제거 후 학습 데이터셋 : 407
중복 제거 후 테스트 데이터셋 : 102


In [6]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification, TFElectraForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# KcELECTRA 모델 및 토크나이저 로드
Kc_model = "beomi/KcELECTRA-base"
Kc_tokenizer = AutoTokenizer.from_pretrained(Kc_model)

c:\Users\GJAISCHOOL\.conda\envs\nv_decem\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
train_data

,Sentence,label
79,"우리나라였으면 남성1년 집해유예, 엄마 20년 때릴것같네요. 아님말구...",0
317,저 양반은 어쩌다 저리 됐을까요?,0
486,물도 꼭 챙겨마셔라 안마시면 죽더라 진짜임,0
397,대중이 까는거 무적권 ㅇㅂ,0
167,일단은 표현의 자유로 남겨놔야하는 것 같음,0
...,...,...
442,연분9등법은,0
438,지금의 정부는 다 계획이 있구나~,0
337,씨발년아 웃겨?,1
236,ㅅㅂ 인구가 준다는데 수도권에 더 많이 살아. 좀 갈사람은 나가라. 특히 586세대들,1


In [8]:
tokenized_train_sentences = Kc_tokenizer( 
    list(train_data["Sentence"]),           # train_data 리스트화
    return_tensors="pt",                    # pytorch의 tensor 형태로 변환
    max_length=128,                         # 최대 토큰길이 설정
    padding=True,                           # 학습 시 빈 값을 0으로 대체
    truncation=True,                        # 초과 토큰 제거
    add_special_tokens=True                 # 추가 토큰 설정
)

In [9]:
tokenized_test_sentences = Kc_tokenizer(
    list(test_data["Sentence"]),           # test_data 리스트화
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [10]:
"""
CLS(분류 토큰)   : 시작을 알릴 때 사용
SEP(구분자 토큰) : 서로 다를 때 구분할 사용할 토큰
MASK            : 특정 토큰이 마스크화, 모델이 예측하도록 훈련
PAD             : 일관된 길이로 동일하게 만듦
                                                            """

'\nCLS(분류 토큰)   : 시작을 알릴 때 사용\nSEP(구분자 토큰) : 서로 다를 때 구분할 사용할 토큰\nMASK            : 특정 토큰이 마스크화, 모델이 예측하도록 훈련\nPAD             : 일관된 길이로 동일하게 만듦\n                                                            '

In [11]:
print(tokenized_train_sentences[0])        # 0번째 문장에 해당하는 객체
print("="*200) 
print(tokenized_train_sentences[0].tokens) # 0번째 문장에 토큰의 목록
print("="*200)
print(tokenized_train_sentences[0].ids)    # 0번째 문장에 대한 고유 ID

Encoding(num_tokens=110, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['Ġìļ°ë¦¬ëĤĺëĿ¼', 'ìĺĢìľ¼ë©´', 'ĠëĤ¨ìĦ±', '1', 'ëħĦ', 'Ġì§ĳ', 'íķ´', 'ìľł', 'ìĺĪ', ',', 'ĠìĹĦë§Ī', 'Ġ20', 'ëħĦ', 'ĠëķĮë¦´', 'ê²ĥê°ĻëĦ¤ìļĶ', '.', 'ĠìķĦëĭĺ', 'ë§Ĳ', 'êµ¬', '...', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [12]:
class CurseDataset(torch.utils.data.Dataset):                                        # 학습을 위한 데이터셋 만들기     
    def __init__(self, encoding, labels):                                            # pytorch에서 학습할 수 있게 데이터셋 생성
        self.encodings = encoding                                                    # Feature Data / 'Sentence'
        self.labels = labels                                                         # Target Data / 'Label'
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # self.encodings 딕셔너리 내의 값 중에 val를 torch.tensor로 변환해 하여
        item["labels"] = torch.tensor(self.labels[idx])                             # key: toch.tensor(val[ix])라는 item 딕셔너리 형성
        return item                                                                 # 새로운 labels 키 값에 value torch.tensor(self.labels[idx]) 쌍을 생성
    
    def __len__(self):
        return len(self.labels)                                                     # self.labels 길이 반환

In [13]:
# train_set, test_set에 대한 데이터셋을 각각 생성
train_label = train_data["label"].values
test_label = test_data["label"].values

train_dataset = CurseDataset(tokenized_train_sentences, train_label)
test_dataset = CurseDataset(tokenized_test_sentences, test_label)

In [14]:
Kc_model = AutoModelForSequenceClassification.from_pretrained(Kc_model, num_labels=2)   # 사전 학습된 모델 찾아오기
Kc_model.to(device)                                                                     # num_labels 평가지표 확인

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.de

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [15]:
# 3 - 2 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir = './results',           # 학습결과 저장경로
    num_train_epochs=10,                # 학습 epoch 설정
    per_device_train_batch_size = 8,    # train batch_size 설정
    per_device_eval_batch_size = 64,    # test batch_size 설정
    logging_dir = './logs',             # 학습 log 저장경로 / 손실 값 저장
    logging_steps=500,                  # 학습 log 기록 단위 / 500번 마다 출력
    save_total_limit = 2,               # 학습결과 저장 최대갯수
)

In [16]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# 학습과정에서 사용할 평가지표를 위한 함수 설정
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  # 정밀도, 재현율, f1 구하기 
  precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
  # 정확도 구하기
  acc = accuracy_score(labels, preds)
  return{
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
  }

In [17]:
# Trainer 모듈을 사용해 모델의 학습을 컨트롤하은 trainer를 생성
trainer = Trainer(
    model = Kc_model,                    # 학습하고자하는 Transformers model
    args=training_args,                  # 위에서 정의한 학습 피라미터 설정
    train_dataset=train_dataset,         # 학습 데이터셋
    eval_dataset=test_dataset,           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 평가지표
)

In [18]:
print(type(trainer))

<class 'transformers.trainer.Trainer'>


In [19]:
trainer.train()

c:\Users\GJAISCHOOL\.conda\envs\nv_decem\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/510 [00:00<?, ?it/s]C:\Users\GJAISCHOOL\AppData\Local\Temp\ipykernel_4536\1869798269.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} # self.encodings 딕셔너리 내의 값 중에 val를 torch.tensor로 변환해 하여
 18%|█▊        | 91/510 [06:23<25:54,  3.71s/it]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Evaluate on the test dataset
results = trainer.predict(test_dataset)

# Get the true labels and predicted logits
true_labels = results.label_ids
predicted_logits = results.predictions.argmax(-1)

# Calculate the confusion matrix
conf_mat = confusion_matrix(true_labels, predicted_logits)

# Plot the confusion matrix using seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=['P', 'N'], yticklabels=['T', 'F'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
def sentence_predict(sent):
    Kc_model.eval()

    Kc_tokenized_sent = Kc_tokenizer(
        text_target=sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )

    Kc_tokenized_sent.to(device)

    with torch.no_grad():
        outputs = Kc_model(
        input_ids=Kc_tokenized_sent["input_ids"],
        attention_mask=Kc_tokenized_sent["attention_mask"],
        token_type_ids=Kc_tokenized_sent["token_type_ids"]
    )

    logits = outputs[0]
    logits = logits.detach().cpu()
    result = logits.argmax(-1)
    if result == 0:
        return False, sent
    elif result == 1:
        return True, sent


In [ ]:
print(sentence_predict("화이팅"))
print(sentence_predict("이게 맞아?"))

In [ ]:
# 'RNN_2' 모듈을 가져오기 위한 경로를 sys.path에 추가
import sys
sys.path.append(r'C:\Users\administrator\Desktop\X_filter\Algorithm\RNN')
 
# 'RNN' 모듈을 import
from RNN import *

In [ ]:
def badword_find(sent):
    badword_df = pd.read_excel(r'C:\Users\GJAISCHOOL\Desktop\X_filter\Algorithm\dataset\word_list (완성).xlsx')
    found_bad_word = False  # 입력 문장에 단어가 발견되었는지를 나타내는 플래그
    for idx, row in badword_df.iterrows():
        if row["WORD"] in sent:
            # 'WORD'가 입력 문장에 포함된 경우
            new_word = row["대체어"]
            if not pd.isnull(new_word):
                # RNN_result = sentence_generation(new_word, 2) + " "
                sent = sent.replace(row["WORD"], new_word)
                found_bad_word = True
            else:
                sent = sent.replace(row["WORD"], "*" * len(row["WORD"]))
                found_bad_word = True
    
    if not found_bad_word:
        # result = "@" * len(input_sentence)
        sent = "혐오 표현입니다."
    return sent

# 테스트
input_sentence = "씨발"
speak_result = badword_find(input_sentence)
print(speak_result)

In [ ]:
def speak_pre(sent):
    is_malicious, _ = sentence_predict(sent)
    if not is_malicious:
        return sent
    else:
        return badword_find(sent)

def speak(sent):
    is_malicious, sent_result = sentence_predict(sent)
    if not is_malicious:
        return sent
    else:
        return badword_find(sent_result)

In [ ]:
import re
import kss
from pykospacing import Spacing
from soynlp.normalizer import repeat_normalize

# 띄어쓰기 설정
spacing = Spacing()

In [ ]:
# 특수문자 제거
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ|a-zA-Z0-9]','', text)
    return text

In [ ]:
def clean_and_repeat_normalize(text):
    cleansed_text = cleanse(text)                                             # 특수문자 제거
    normalized_text = repeat_normalize(cleansed_text, num_repeats=2)          # 중복문자 제거
    input_data = re.sub(r'\d', '', normalized_text)                           # 제거
    normalized_text = spacing(input_data)                                     # 띄어쓰기 보정 

    return normalized_text

clean_and_repeat_normalize("아버지가방에들어가신다")

In [ ]:
def final_output():
    input_text = clean_and_repeat_normalize(input("텍스트를 입력하세요: "))
    sentences = kss.split_sentences(input_text)

    sentences_list = []
    for sentence in sentences:
        result = speak(sentence)
        sentences_list.append(result) 

    long_test = ' '.join(sentences_list)
    return long_test

In [ ]:
# 시발 정작 지네들 자식은 몸쓰는일 안시킬거면서 개지랄떠네 ㅋㅋㅋ
final_output()

In [ ]:
import subprocess
result_1 = subprocess.run(['pip', 'install', 'tensorflow'], capture_output=True, text=True)   # 필요 라이브러리 설치
result_2 = subprocess.run(['pip', 'show', 'tensorflow'], capture_output=True, text=True)
print(result_1.stdout)
print(result_2.stdout)                                               

import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df = pd.read_csv(r'C:\Users\GJAISCHOOL\Desktop\sentence pl data1113 수정.csv')

sentences = df['Sentence'].tolist()
rnn_tokenizer = Tokenizer()
rnn_tokenizer.fit_on_texts(sentences)

vocab_size = len(rnn_tokenizer.word_index) + 1                          # 어휘 사전에 크기 계산 

sequences = list()                                                      # 각 문장을 단어 시퀀스로 변환하고, 문장의 부분 시퀀스를 생성하여 리스트에 추가
for line in sentences:
    encoded = rnn_tokenizer.texts_to_sequences([line])[0]               # 현재 문장을 정수로 시퀀스로 변수
    for i in range(1, len(encoded)):                                    # 문장의 부분 시퀀스를 생성하고 리스트에 추가
        sequence = encoded[: i+1]
        sequences.append(sequence)

max_len = max(len(l) for l in sequences)                                # 모든 샘플에서 길이가 가장 긴 샘플의 길이 출력
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')     # 시퀀스 데이터를 패딩하여 일정한 길이로 만듦 (여기서는 가장 긴 샘플의 길이로 패딩)

sequences = np.array(sequences)                                         # 패딩이 된 시퀀스를 넘파이 배열로 변환
                                                                        # 입력 및 출력 데이터 생성
X = sequences[:, :-1]                                                   # 시퀀스에서 마지막 단어를 제외한 부분은 입력(X)
y = sequences[:, -1]                                                    # 시퀀스에 마지막 단어는 출력
y = to_categorical(y, num_classes=vocab_size)                           # 출력(y)을 one-hot 인코딩 형태로 변환

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

embedding_dim = 10                                                      # 임베딩 차원을 10으로 10차원의 실수 벡터로 표현
hidden_units = 32                                                       # 순환 신경망 내 내부에서의 뉴련의 수

RNN_model = Sequential()                                                    
RNN_model.add(Embedding(vocab_size, embedding_dim))                     # Embedding 레이어 추가: 어휘 사전의 크기를 입력으로 받고, 지정된 임베딩 차원으로 단어를 임베딩
RNN_model.add(SimpleRNN(hidden_units))                                  # 지정된 은닉 유닛 수를 가진 단순한 RNN 레이어
RNN_model.add(Dense(vocab_size, activation='softmax'))                  # 출력 레이어 추가 : 어휘 사전의 크길르 가진 Dense 레이어를 사용하며, 활성화 함수는 softmax
RNN_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
                                                                        # 모델 컴파일: 다중 클래스 분류 문제이므로 categorical_crossentropy 손실 함수와 Adam 옵티마이저 사용
RNN_model.fit(X, y, epochs=20, verbose=2)                               # 모델 학습: 입력(X) 및 출력(y) 데이터를 사용하여 주어진 에폭 동안 모델을 학습

def sentence_generation(current_word, n, temperature=1.0, model=RNN_model, tokenizer=rnn_tokenizer):
    init_word = current_word                                            # 시작 단어와 문장을 저장할 빈 문자열 초기화
    sentence = ''

    for _ in range(n):                                                  # n개 단어 만큼 문자 생성
        encoded = rnn_tokenizer.texts_to_sequences([current_word])[0]   # 현재 단어를 토크나이저를 사용하여 정수 시퀀스로 변환 
                                                                        # ex) "Hello, how are you?" => "Hello"가 1, "how"가 2, "are"가 3, "you"가 4라고 할 때 => [1, 2, 3, 4]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')     # 시퀀스가 5로 설정하고, 부족하면 앞쪽에 0을 채워넣고 부족하면 앞부분을 자른다.
                                                                        # 만약 뒤쪽으로 바꾸고싶으면 padding='post를 주면 된다.
        result = RNN_model.predict(encoded, verbose=0)                  # 모델의 다음단어 예측
        result = result / temperature                                   # 확률에 온도를 적용하여 랜덤성 조절
        result = np.exp(result) / np.sum(np.exp(result), axis=1)        # 소프트맥스를 적용하여 최종 확률 얻기
        sampled_index = np.random.choice(len(result[0]), p=result[0])   # 예측된 확률을 기반으로 다음 단어 샘플링
        word = rnn_tokenizer.index_word[sampled_index]

        current_word = current_word + ' ' + word                        # 현재 단어와 문장 업데이트
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence